In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt

client_train = pd.read_csv("/content/drive/MyDrive/MMAI869_TeamAssignment/train/client_train.csv",low_memory=False)
invoice_train = pd.read_csv("/content/drive/MyDrive/MMAI869_TeamAssignment/train/invoice_train.csv",low_memory=False)

client_test = pd.read_csv("/content/drive/MyDrive/MMAI869_TeamAssignment/test/client_test.csv",low_memory=False)
invoice_test = pd.read_csv("/content/drive/MyDrive/MMAI869_TeamAssignment/test/invoice_test.csv", low_memory=False)

In [3]:
for df in [invoice_train,invoice_test]:
    df['invoice_date'] = pd.to_datetime(df['invoice_date'])

In [4]:
invoice_train.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,ELEC


In [5]:
d={"ELEC":0,"GAZ":1}
invoice_train['counter_type']=invoice_train['counter_type'].map(d)
invoice_test['counter_type']=invoice_test['counter_type'].map(d)

In [ ]:
invoice_train.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,0,0,14302,14384,4,0
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,0,0,12294,13678,4,0
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,0,0,14624,14747,4,0
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,0,0,14747,14849,4,0
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,0,0,15066,15638,12,0


In [ ]:
client_train.head()

,disrict,client_id,client_catg,region,creation_date,target
0,60,train_Client_0,11,101,31/12/1994,0.0
1,69,train_Client_1,11,107,29/05/2002,0.0
2,62,train_Client_10,11,301,13/03/1986,0.0
3,69,train_Client_100,11,105,11/07/1996,0.0
4,62,train_Client_1000,11,303,14/10/2014,0.0


In [6]:
client_train['client_catg'] = client_train['client_catg'].astype(int)
client_train['disrict'] = client_train['disrict'].astype(int)

client_test['client_catg'] = client_test['client_catg'].astype(int)
client_test['disrict'] = client_test['disrict'].astype(int)

In [7]:
# @markdown Aggregate the client dataset with the invoice dataset

def aggregate_by_client_id(invoice_data):
    aggs = {}
    aggs['consommation_level_1'] = ['mean']
    aggs['consommation_level_2'] = ['mean']
    aggs['consommation_level_3'] = ['mean']
    aggs['consommation_level_4'] = ['mean']

    agg_trans = invoice_data.groupby(['client_id']).agg(aggs)
    agg_trans.columns = ['_'.join(col).strip() for col in agg_trans.columns.values]
    agg_trans.reset_index(inplace=True)

    df = (invoice_data.groupby('client_id')
            .size()
            .reset_index(name='{}transactions_count'.format('1')))
    return pd.merge(df, agg_trans, on='client_id', how='left')

In [8]:
agg_train = aggregate_by_client_id(invoice_train)

In [ ]:
agg_train.head()

,client_id,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,train_Client_0,35,352.400000,10.571429,0.000000,0.000000
1,train_Client_1,37,557.540541,0.000000,0.000000,0.000000
2,train_Client_10,18,798.611111,37.888889,0.000000,0.000000
3,train_Client_100,20,1.200000,0.000000,0.000000,0.000000
4,train_Client_1000,14,663.714286,104.857143,117.357143,36.714286


In [9]:
train = pd.merge(client_train,agg_train, on='client_id', how='left')

In [ ]:
train.head()

,disrict,client_id,client_catg,region,creation_date,target,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,60,train_Client_0,11,101,31/12/1994,0.0,35,352.400000,10.571429,0.000000,0.000000
1,69,train_Client_1,11,107,29/05/2002,0.0,37,557.540541,0.000000,0.000000,0.000000
2,62,train_Client_10,11,301,13/03/1986,0.0,18,798.611111,37.888889,0.000000,0.000000
3,69,train_Client_100,11,105,11/07/1996,0.0,20,1.200000,0.000000,0.000000,0.000000
4,62,train_Client_1000,11,303,14/10/2014,0.0,14,663.714286,104.857143,117.357143,36.714286


In [10]:
agg_test = aggregate_by_client_id(invoice_test)
test = pd.merge(client_test,agg_test, on='client_id', how='left')

In [ ]:
test.head()

,disrict,client_id,client_catg,region,creation_date,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,62,test_Client_0,11,307,28/05/2002,37,488.135135,3.243243,0.000000,0.000000
1,69,test_Client_1,11,103,06/08/2009,22,1091.409091,843.136364,182.318182,586.318182
2,62,test_Client_10,11,310,07/04/2004,74,554.040541,37.364865,15.743243,0.162162
3,60,test_Client_100,11,101,08/10/1992,40,244.350000,0.000000,0.000000,0.000000
4,62,test_Client_1000,11,301,21/07/1977,53,568.188679,145.056604,33.679245,0.000000


In [11]:
sub_client_id = test['client_id']
drop_columns = ['client_id', 'creation_date']

for col in drop_columns:
    if col in train.columns:
        train.drop([col], axis=1, inplace=True)
    if col in test.columns:
        test.drop([col], axis=1, inplace=True)

In [ ]:
train.head()

,disrict,client_catg,region,target,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,60,11,101,0.0,35,352.400000,10.571429,0.000000,0.000000
1,69,11,107,0.0,37,557.540541,0.000000,0.000000,0.000000
2,62,11,301,0.0,18,798.611111,37.888889,0.000000,0.000000
3,69,11,105,0.0,20,1.200000,0.000000,0.000000,0.000000
4,62,11,303,0.0,14,663.714286,104.857143,117.357143,36.714286


In [ ]:
test.head()

,disrict,client_catg,region,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,62,11,307,37,488.135135,3.243243,0.000000,0.000000
1,69,11,103,22,1091.409091,843.136364,182.318182,586.318182
2,62,11,310,74,554.040541,37.364865,15.743243,0.162162
3,60,11,101,40,244.350000,0.000000,0.000000,0.000000
4,62,11,301,53,568.188679,145.056604,33.679245,0.000000


In [12]:
train.to_csv("/content/drive/MyDrive/MMAI869_TeamAssignment/train/preProTrain.csv")

In [13]:
test.to_csv("/content/drive/MyDrive/MMAI869_TeamAssignment/test/preproTest.csv")

In [34]:
from xgboost import XGBClassifier

x_train = train.drop(columns=['target'])
y_train = train['target']

model = XGBClassifier(
                      learning_rate =0.2,
                      n_estimators=1000,
                      max_depth=10,
                      min_child_weight=4,
                      gamma=0,
                      subsample=0.8,
                      colsample_bytree=0.8,
                      objective= 'binary:logistic',
                      nthread=4,
                      scale_pos_weight=1,
                      seed=27)

"""
      NO DATA PRE-PROCESSING AND FEATURE ENGINEERING
     Score 0.50966   max_dept = 5 min_child_weight = 1
     Score 0.51023   max_dept = 8, min_child_weight = 4  (12 sec run time) the run time increases as I increases the maxdept&min_child
     Score 0.51121            = 10,                 = 6
     Score 0.51467            = 10,                 = 4  learning rate 0.2 (13 sec run time)
"""
model.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=4, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None, nthread=4,
              num_parallel_tree=None, ...)

In [ ]:
x_train.head()

,disrict,client_catg,region,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,60,11,101,35,352.400000,10.571429,0.000000,0.000000
1,69,11,107,37,557.540541,0.000000,0.000000,0.000000
2,62,11,301,18,798.611111,37.888889,0.000000,0.000000
3,69,11,105,20,1.200000,0.000000,0.000000,0.000000
4,62,11,303,14,663.714286,104.857143,117.357143,36.714286


In [ ]:
y_train.head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: target, dtype: float64

In [ ]:
test.head()

,disrict,client_catg,region,1transactions_count,consommation_level_1_mean,consommation_level_2_mean,consommation_level_3_mean,consommation_level_4_mean
0,62,11,307,37,488.135135,3.243243,0.000000,0.000000
1,69,11,103,22,1091.409091,843.136364,182.318182,586.318182
2,62,11,310,74,554.040541,37.364865,15.743243,0.162162
3,60,11,101,40,244.350000,0.000000,0.000000,0.000000
4,62,11,301,53,568.188679,145.056604,33.679245,0.000000


In [35]:
pred = model.predict(test)
preds = pd.DataFrame(pred, columns=['target'])

In [21]:
print(pred)

[0 0 0 ... 0 0 0]


In [36]:
submission = pd.DataFrame(
    {
        'client_id': sub_client_id,
        'target': preds['target']
    }
)

submission.head()

,client_id,target
0,test_Client_0,0
1,test_Client_1,0
2,test_Client_10,0
3,test_Client_100,0
4,test_Client_1000,0


In [37]:
submission.to_csv("/content/drive/MyDrive/MMAI869_TeamAssignment/submission4.csv",index=False)

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
nbModel = gnb.fit(x_train,y_train)

nbPred = nbModel.predict(test)

In [ ]:
print(nbPred)

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
nbPreds = pd.DataFrame(nbPred, columns=['target'])

In [ ]:
nbPreds.head()

,target
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [ ]:
submission = pd.DataFrame(
    {
        'client_id': sub_client_id,
        'target': nbPreds['target']
    }
)

submission.head()

,client_id,target
0,test_Client_0,0.0
1,test_Client_1,0.0
2,test_Client_10,0.0
3,test_Client_100,0.0
4,test_Client_1000,0.0


In [ ]:
submission.to_csv("/content/drive/MyDrive/MMAI869_TeamAssignment/submission_nbModel.csv",index=False)

In [ ]:
from sklearn import svm

svmClassifier = svm.SVC()
svmModel = svmClassifier.fit(x_train,y_train)

In [ ]:
svmPred = svmModel.predict(test)

In [ ]:
print(svmPred)

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
svmPreds = pd.DataFrame(svmPred, columns=['target'])

In [ ]:
svmSubmission = pd.DataFrame(
    {
        'client_id': sub_client_id,
        'target': svmPreds['target']
    }
)

svmSubmission.head()

,client_id,target
0,test_Client_0,0.0
1,test_Client_1,0.0
2,test_Client_10,0.0
3,test_Client_100,0.0
4,test_Client_1000,0.0


In [ ]:
svmSubmission.to_csv("/content/drive/MyDrive/MMAI869_TeamAssignment/submission_svmModel.csv",index=False)